In [1]:
import pandas as pd

In [2]:
from wikidata.client import WikiDataClient

In [3]:
countries_query =  \
"""
SELECT ?item ?itemLabel ?population
WHERE
{
  ?item wdt:P31 wd:Q6256.
  ?item wdt:P1082 ?population.
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} 
ORDER BY DESC(?population)
"""

In [4]:
df = WikiDataClient.query(countries_query)
df

,item,population,itemLabel
0,http://www.wikidata.org/entity/Q148,1409517397,People's Republic of China
1,http://www.wikidata.org/entity/Q668,1349217956,India
2,http://www.wikidata.org/entity/Q30,325145963,United States of America
3,http://www.wikidata.org/entity/Q252,263991379,Indonesia
4,http://www.wikidata.org/entity/Q155,208494900,Brazil
...,...,...,...
156,http://www.wikidata.org/entity/Q695,20918,Palau
157,http://www.wikidata.org/entity/Q697,13650,Nauru
158,http://www.wikidata.org/entity/Q672,9876,Tuvalu
159,http://www.wikidata.org/entity/Q34020,1612,Niue


In [5]:
df.population.dtype

dtype('int64')